In [2]:
import numpy as np

import pandas as pd
import os
import glob
from matplotlib import pyplot
import yaml

In [95]:
def read_analysis(path):
    with open(f"{path}/mnistc-analysis.yml", "r") as fh:
        analysis = yaml.safe_load(fh)

    with open(f"{path}/summary.yml", "r") as fh:
        summary = yaml.safe_load(fh)
    name = path.split("/")[-1]


    arch = summary["model"].split("/")[0]
    
    key = "-".join([arch, summary["strategy"], summary["class_loss"], summary["cov_type"], "M" + str(summary["M"])])

    rows = []

    for cat in analysis["categories"]:
        for r in analysis["categories"][cat]:
            if r["L"] == 12:                
                rows.append(dict(
                    name=name,
                    clean_accuracy=summary["metrics"]["test"]["accuracy_L12"],
                    category=cat,
                    accuracy=r["accuracy"],
                    key=key,
                    model=arch,
                    strategy=summary["strategy"],
                    class_loss=summary["class_loss"],
                    cov_type=summary["cov_type"],
                    M=summary["M"],
                    L=r["L"],
                    beta=summary["beta"]
                ))

    return rows
       
read_analysis("../artifacts-server/hydra/experiment-mnist-256d-test-robustness/mnist-various-opts-M1/mlp-vdb-diag-mnist--2019-12-21--14-36-54-477728/")

[{'name': '',
  'clean_accuracy': 0.984700083732605,
  'category': 'brightness',
  'accuracy': 0.02669999934732914,
  'key': 'mlp-algo1/d:5|e:1-vdb-diag-M1',
  'model': 'mlp',
  'strategy': 'algo1/d:5|e:1',
  'class_loss': 'vdb',
  'cov_type': 'diag',
  'M': 1,
  'L': 12,
  'beta': 0.001},
 {'name': '',
  'clean_accuracy': 0.984700083732605,
  'category': 'canny_edges',
  'accuracy': 0.5590000152587891,
  'key': 'mlp-algo1/d:5|e:1-vdb-diag-M1',
  'model': 'mlp',
  'strategy': 'algo1/d:5|e:1',
  'class_loss': 'vdb',
  'cov_type': 'diag',
  'M': 1,
  'L': 12,
  'beta': 0.001},
 {'name': '',
  'clean_accuracy': 0.984700083732605,
  'category': 'dotted_line',
  'accuracy': 0.971500039100647,
  'key': 'mlp-algo1/d:5|e:1-vdb-diag-M1',
  'model': 'mlp',
  'strategy': 'algo1/d:5|e:1',
  'class_loss': 'vdb',
  'cov_type': 'diag',
  'M': 1,
  'L': 12,
  'beta': 0.001},
 {'name': '',
  'clean_accuracy': 0.984700083732605,
  'category': 'fog',
  'accuracy': 0.5449000000953674,
  'key': 'mlp-algo1/

In [12]:
def read_experiment_metrics(directory, strategies=["oneshot", "algo2/k:5", "algo2/k:10", "algo2/k:20"], M=[1, 6, 12]):
    models = glob.glob(f"{directory}/summary.yml")
    
    models = list(map(lambda x: os.path.dirname(x), models))
    
    rows = []
    
    for m in models:
        
        rows.extend(read_analysis(m))
        
    df = pd.DataFrame(rows)
    df["error"] = (1-df["accuracy"])
    return df[df.M.isin(M) & df.strategy.isin(strategies)].copy()
    
df_mnist_vdb_256d = read_experiment_metrics(
    "../artifacts-server/hydra/experiment-mnist-256d-test-robustness/*/*"
)

# df_mnist_vib_256d = read_experiment_metrics(
#     "../artifacts-server/hydra/experiment-mnist-256d-test-robustness-vib/*/*"
# )

In [92]:
mnistc_categories = """
shot_noise
impulse_noise
glass_blur
motion_blur
shear
scale
rotate
brightness
translate
stripe
fog
spatter
dotted_line
canny_edges
zigzag
""".strip().split("\n")

def print_table(df, category, should_print=False):
    
    df = df[df.category == category]\
        .groupby(["strategy", "M"])\
        .agg(error_mean=("error", "mean"), error_std=("error", "std")) \
        .reset_index() 
    
    df["temp"] = df[['error_mean', 'error_std']].apply(lambda x: "%.2f±%.4f\%%" % (x[0]*100, x[1]*100), axis=1) 
    
    
    df = df.drop(columns=["error_mean", "error_std"])\
        .rename(columns={"temp": f"{category}-error"}) \
        .pivot(index="strategy", columns="M") \
        .sort_values(by="strategy", ascending=False)
    
    if should_print:
        print("")
        print(df.to_latex(escape=False))
        print("")
    
    return df


for c in mnistc_categories:
    print(f"Category {c}")
    print_table(df_mnist_vdb_256d, category=c, should_print=True)

Category shot_noise

\begin{tabular}{llll}
\toprule
{} & \multicolumn{3}{l}{shot_noise-error} \\
M &               1  &             6  &             12 \\
strategy   &                  &                &                \\
\midrule
oneshot    &    3.01±0.1905\% &  2.45±0.1974\% &  2.39±0.1870\% \\
algo2/k:5  &    3.13±0.1748\% &  2.93±1.1526\% &  2.40±0.1531\% \\
algo2/k:20 &    2.83±0.5167\% &  2.39±0.1940\% &  2.38±0.1034\% \\
algo2/k:10 &    3.01±0.6208\% &  2.46±0.0535\% &  2.34±0.1176\% \\
\bottomrule
\end{tabular}


Category impulse_noise

\begin{tabular}{llll}
\toprule
{} & \multicolumn{3}{l}{impulse_noise-error} \\
M &                  1  &             6  &             12 \\
strategy   &                     &                &                \\
\midrule
oneshot    &       5.93±0.8684\% &  3.31±0.1545\% &  3.39±0.1247\% \\
algo2/k:5  &       4.37±0.5605\% &  3.61±0.7436\% &  3.49±0.0411\% \\
algo2/k:20 &       3.77±0.4492\% &  3.20±0.2087\% &  3.27±0.1284\% \\
algo2/k:10 &       3

In [105]:
for c in mnistc_categories:
    print(f"Category {c}")
    print_table(df_mnist_vib_256d, category=c, should_print=True)

Category shot_noise

\begin{tabular}{lll}
\toprule
{} & \multicolumn{2}{l}{shot_noise-error} \\
M &               6  &             12 \\
strategy &                  &                \\
\midrule
oneshot  &    2.52±0.1072\% &  2.38±0.0826\% \\
\bottomrule
\end{tabular}


Category impulse_noise

\begin{tabular}{lll}
\toprule
{} & \multicolumn{2}{l}{impulse_noise-error} \\
M &                  6  &             12 \\
strategy &                     &                \\
\midrule
oneshot  &       3.20±0.1323\% &  3.19±0.1186\% \\
\bottomrule
\end{tabular}


Category glass_blur

\begin{tabular}{lll}
\toprule
{} & \multicolumn{2}{l}{glass_blur-error} \\
M &               6  &              12 \\
strategy &                  &                 \\
\midrule
oneshot  &   19.69±3.9325\% &  16.94±1.1496\% \\
\bottomrule
\end{tabular}


Category motion_blur

\begin{tabular}{lll}
\toprule
{} & \multicolumn{2}{l}{motion_blur-error} \\
M &                6  &              12 \\
strategy &                   & 

# mCE and Rel CE

In [121]:
def compute_corruption_error(df, ref_key, strategies=["oneshot", "algo2/k:10"]):
    group_key = ["key", "category"]

    df = df[df.strategy.isin(strategies) & df.category.isin(mnistc_categories)].copy()
    
    df["error"] = 100*(1-df["accuracy"])
    df["error_clean"] = 100*(1-df["clean_accuracy"])

    df_agg_err = df[df.category == "brightness"]\
        .groupby(["key"])\
        .agg(mean_err=("error_clean", "mean"), std_err=("error_clean", "std"))\
        .reset_index()
    
    df_agg_err["mean_std"] = df_agg_err[["mean_err", "std_err"]].apply(lambda x: "%.2f±%.2f\%%" % (x[0], x[1]), axis=1) 


    # calculate relative_error for each run
    df["rel_error"] = df["error"] - df["error_clean"]


    # sum reference metrics over all runs 
    df_ref_setting = df[df.key == ref_key].groupby(group_key)\
        .agg(ref_sum_error=("error", "sum"), ref_rel_sum_error=("rel_error", "sum")) \
        .reset_index()
    
    
    # sun metrics over all runs
    df_other_models = df.groupby(group_key) \
        .agg(sum_error=("error", "sum"), rel_sum_error=("rel_error", "sum")) \
        .reset_index()


    df_result = df_other_models.merge(df_ref_setting[["category", "ref_sum_error", "ref_rel_sum_error"]], on="category")
    
    # compute CE, rel CE per category
    df_result["ce"] = 100*df_result["sum_error"] / df_result["ref_sum_error"]
    df_result["rel_ce"] = 100* df_result["rel_sum_error"] / df_result["ref_rel_sum_error"]

    
    return df_result.groupby(["key"])\
        .agg(
            mCe=("ce", "mean"),
            rel_mCe=("rel_ce", "mean"), 
        ).merge(df_agg_err[["key", "mean_std"]], on="key")[["key", "mean_std", "mCe", "rel_mCe"]]

df_temp2 = compute_corruption_error(
    pd.concat([mnist_beta03, mnist_beta03_vib], ignore_index=True, sort=False),
    "mlp-oneshot-vdb-diag-M1",
    strategies=["oneshot", "algo2/k:5", "algo2/k:10", "algo2/k:20"]
)

df_temp2

,key,mean_std,mCe,rel_mCe
0,mlp-algo2/k:10-vdb-diag-M1,1.42±0.13\%,93.878304,93.556227
1,mlp-algo2/k:10-vdb-diag-M12,1.28±0.06\%,85.846514,85.281419
2,mlp-algo2/k:10-vdb-diag-M6,1.39±0.04\%,85.949839,83.385418
3,mlp-algo2/k:20-vdb-diag-M1,1.49±0.16\%,95.417799,92.836762
4,mlp-algo2/k:20-vdb-diag-M12,1.37±0.09\%,84.934681,82.657614
5,mlp-algo2/k:20-vdb-diag-M6,1.37±0.10\%,86.371412,83.843686
6,mlp-algo2/k:5-vdb-diag-M1,1.52±0.05\%,98.600738,98.044251
7,mlp-algo2/k:5-vdb-diag-M12,1.38±0.09\%,86.315594,83.905726
8,mlp-algo2/k:5-vdb-diag-M6,1.65±0.43\%,91.901023,87.589317
9,mlp-oneshot-vdb-diag-M1,1.53±0.10\%,100.000000,100.000000


In [126]:
(1-mnist_beta03[ (mnist_beta03.key=="mlp-oneshot-vdb-diag-M1") &  (mnist_beta03.category =="brightness")].clean_accuracy).mean()

0.015324816107749939

In [127]:
# print_table(df_mnist_vdb_256d, category="identity", should_print=True)


\begin{tabular}{llll}
\toprule
{} & \multicolumn{3}{l}{identity-error} \\
M &             1  &             6  &             12 \\
strategy   &                &                &                \\
\midrule
oneshot    &  1.54±0.1773\% &  1.41±0.0556\% &  1.32±0.0311\% \\
algo2/k:5  &  1.50±0.0455\% &  1.56±0.4110\% &  1.39±0.0668\% \\
algo2/k:20 &  1.46±0.1606\% &  1.33±0.0900\% &  1.37±0.0806\% \\
algo2/k:10 &  1.43±0.0889\% &  1.39±0.0638\% &  1.30±0.0408\% \\
\bottomrule
\end{tabular}




identity-error                              
M                      1              6              12
strategy                                               
oneshot     1.54±0.1773\%  1.41±0.0556\%  1.32±0.0311\%
algo2/k:5   1.50±0.0455\%  1.56±0.4110\%  1.39±0.0668\%
algo2/k:20  1.46±0.1606\%  1.33±0.0900\%  1.37±0.0806\%
algo2/k:10  1.43±0.0889\%  1.39±0.0638\%  1.30±0.0408\%

In [78]:
# mnist_beta02 = read_experiment_metrics("../artifacts-server/mpg-server/experiment-mnist-256d-test-robustness-beta02/*/*")

mnist_beta03 = read_experiment_metrics("../artifacts-server/hydra/experiment-mnist-256d-test-robustness/*/*")
mnist_beta03_vib = read_experiment_metrics("../artifacts-server/hydra/experiment-mnist-256d-test-robustness-vib/*/*")

In [134]:
def table_beta(p1, p2):
    df_1, df_2 = read_experiment_metrics(p1), read_experiment_metrics(p2)
    beta_1, beta_2 = df_1.beta.unique(), df_2.beta.unique()

    assert (len(beta_2) == len(beta_1) == 1) and beta_1[0] == beta_2[0]
    print("beta", beta_1[0])
    df = compute_corruption_error(
        pd.concat([df_1, df_2], ignore_index=True, sort=False),
        "mlp-oneshot-vdb-diag-M1",
        strategies=["oneshot", "algo2/k:5", "algo2/k:10", "algo2/k:20"]
    )
    
    with pd.option_context('display.float_format', '{:0.2f}'.format):

        print(df.to_latex(escape=False))
    
    return df

table_beta(
    "../artifacts-server/hydra/experiment-mnist-256d-test-robustness/*/*",
    "../artifacts-server/hydra/experiment-mnist-256d-test-robustness-vib/*/*"
)

beta 0.001
\begin{tabular}{lllrr}
\toprule
{} &                          key &     mean_std &    mCe &  rel_mCe \\
\midrule
0  &   mlp-algo2/k:10-vdb-diag-M1 &  1.42±0.13\% &  93.88 &    93.56 \\
1  &  mlp-algo2/k:10-vdb-diag-M12 &  1.28±0.06\% &  85.85 &    85.28 \\
2  &   mlp-algo2/k:10-vdb-diag-M6 &  1.39±0.04\% &  85.95 &    83.39 \\
3  &   mlp-algo2/k:20-vdb-diag-M1 &  1.49±0.16\% &  95.42 &    92.84 \\
4  &  mlp-algo2/k:20-vdb-diag-M12 &  1.37±0.09\% &  84.93 &    82.66 \\
5  &   mlp-algo2/k:20-vdb-diag-M6 &  1.37±0.10\% &  86.37 &    83.84 \\
6  &    mlp-algo2/k:5-vdb-diag-M1 &  1.52±0.05\% &  98.60 &    98.04 \\
7  &   mlp-algo2/k:5-vdb-diag-M12 &  1.38±0.09\% &  86.32 &    83.91 \\
8  &    mlp-algo2/k:5-vdb-diag-M6 &  1.65±0.43\% &  91.90 &    87.59 \\
9  &      mlp-oneshot-vdb-diag-M1 &  1.53±0.10\% & 100.00 &   100.00 \\
10 &     mlp-oneshot-vdb-diag-M12 &  1.36±0.06\% &  87.29 &    85.36 \\
11 &      mlp-oneshot-vdb-diag-M6 &  1.38±0.04\% &  86.84 &    84.46 \\
12 &     mlp

,key,mean_std,mCe,rel_mCe
0,mlp-algo2/k:10-vdb-diag-M1,1.42±0.13\%,93.878304,93.556227
1,mlp-algo2/k:10-vdb-diag-M12,1.28±0.06\%,85.846514,85.281419
2,mlp-algo2/k:10-vdb-diag-M6,1.39±0.04\%,85.949839,83.385418
3,mlp-algo2/k:20-vdb-diag-M1,1.49±0.16\%,95.417799,92.836762
4,mlp-algo2/k:20-vdb-diag-M12,1.37±0.09\%,84.934681,82.657614
5,mlp-algo2/k:20-vdb-diag-M6,1.37±0.10\%,86.371412,83.843686
6,mlp-algo2/k:5-vdb-diag-M1,1.52±0.05\%,98.600738,98.044251
7,mlp-algo2/k:5-vdb-diag-M12,1.38±0.09\%,86.315594,83.905726
8,mlp-algo2/k:5-vdb-diag-M6,1.65±0.43\%,91.901023,87.589317
9,mlp-oneshot-vdb-diag-M1,1.53±0.10\%,100.000000,100.000000


In [135]:
table_beta(
    "../artifacts-server/mpg-server/experiment-mnist-256d-test-robustness-beta02/*/*",
    "../artifacts-server/mpg-server/experiment-mnist-256d-test-robustness-vib-beta02/*/*"
)

beta 0.01
\begin{tabular}{lllrr}
\toprule
{} &                          key &     mean_std &    mCe &  rel_mCe \\
\midrule
0  &   mlp-algo2/k:10-vdb-diag-M1 &  1.56±0.13\% &  95.58 &    94.74 \\
1  &  mlp-algo2/k:10-vdb-diag-M12 &  1.45±0.06\% &  85.40 &    83.50 \\
2  &   mlp-algo2/k:10-vdb-diag-M6 &  1.52±0.07\% &  89.56 &    88.19 \\
3  &   mlp-algo2/k:20-vdb-diag-M1 &  1.49±0.07\% &  94.44 &    94.38 \\
4  &  mlp-algo2/k:20-vdb-diag-M12 &  1.46±0.06\% &  86.17 &    84.22 \\
5  &   mlp-algo2/k:20-vdb-diag-M6 &  1.47±0.07\% &  87.85 &    86.47 \\
6  &    mlp-algo2/k:5-vdb-diag-M1 &  1.51±0.08\% &  94.66 &    93.86 \\
7  &   mlp-algo2/k:5-vdb-diag-M12 &  1.50±0.09\% &  86.91 &    84.83 \\
8  &    mlp-algo2/k:5-vdb-diag-M6 &  1.44±0.06\% &  89.31 &    88.68 \\
9  &      mlp-oneshot-vdb-diag-M1 &  1.60±0.10\% & 100.00 &   100.00 \\
10 &     mlp-oneshot-vdb-diag-M12 &  1.47±0.13\% &  88.08 &    86.96 \\
11 &      mlp-oneshot-vdb-diag-M6 &  1.51±0.07\% &  89.17 &    87.09 \\
12 &     mlp-

,key,mean_std,mCe,rel_mCe
0,mlp-algo2/k:10-vdb-diag-M1,1.56±0.13\%,95.581273,94.740466
1,mlp-algo2/k:10-vdb-diag-M12,1.45±0.06\%,85.402629,83.495888
2,mlp-algo2/k:10-vdb-diag-M6,1.52±0.07\%,89.564496,88.188118
3,mlp-algo2/k:20-vdb-diag-M1,1.49±0.07\%,94.438817,94.376637
4,mlp-algo2/k:20-vdb-diag-M12,1.46±0.06\%,86.169656,84.215173
5,mlp-algo2/k:20-vdb-diag-M6,1.47±0.07\%,87.851655,86.466720
6,mlp-algo2/k:5-vdb-diag-M1,1.51±0.08\%,94.657912,93.864616
7,mlp-algo2/k:5-vdb-diag-M12,1.50±0.09\%,86.905396,84.834867
8,mlp-algo2/k:5-vdb-diag-M6,1.44±0.06\%,89.313207,88.682038
9,mlp-oneshot-vdb-diag-M1,1.60±0.10\%,100.000000,100.000000


In [131]:
table_beta(
    "../artifacts-server/mpg-server/experiment-mnist-256d-test-robustness-beta04/*/*",
    "../artifacts-server/mpg-server/experiment-mnist-256d-test-robustness-vib-beta04/*/*"
)

beta 0.0001
\begin{tabular}{lllrr}
\toprule
{} &                          key &     mean_std &    mCe &  rel_mCe \\
\midrule
0  &   mlp-algo2/k:10-vdb-diag-M1 &  1.39±0.08\% &  89.64 &    87.80 \\
1  &  mlp-algo2/k:10-vdb-diag-M12 &  1.25±0.07\% &  89.03 &    89.96 \\
2  &   mlp-algo2/k:10-vdb-diag-M6 &  1.32±0.07\% &  88.25 &    86.85 \\
3  &   mlp-algo2/k:20-vdb-diag-M1 &  1.31±0.06\% &  87.34 &    85.71 \\
4  &  mlp-algo2/k:20-vdb-diag-M12 &  1.27±0.06\% &  89.19 &    88.43 \\
5  &   mlp-algo2/k:20-vdb-diag-M6 &  1.28±0.12\% &  87.78 &    87.18 \\
6  &    mlp-algo2/k:5-vdb-diag-M1 &  1.41±0.09\% &  91.71 &    90.06 \\
7  &   mlp-algo2/k:5-vdb-diag-M12 &  1.41±0.20\% &  92.99 &    92.82 \\
8  &    mlp-algo2/k:5-vdb-diag-M6 &  1.27±0.06\% &  85.28 &    84.70 \\
9  &      mlp-oneshot-vdb-diag-M1 &  1.46±0.11\% & 100.00 &   100.00 \\
10 &     mlp-oneshot-vdb-diag-M12 &  1.47±0.30\% & 101.23 &   102.20 \\
11 &      mlp-oneshot-vdb-diag-M6 &  1.30±0.08\% &  91.13 &    90.95 \\
12 &     ml

,key,mean_std,mCe,rel_mCe
0,mlp-algo2/k:10-vdb-diag-M1,1.39±0.08\%,89.638114,87.796430
1,mlp-algo2/k:10-vdb-diag-M12,1.25±0.07\%,89.033382,89.960651
2,mlp-algo2/k:10-vdb-diag-M6,1.32±0.07\%,88.249253,86.853362
3,mlp-algo2/k:20-vdb-diag-M1,1.31±0.06\%,87.339677,85.708763
4,mlp-algo2/k:20-vdb-diag-M12,1.27±0.06\%,89.185653,88.433692
5,mlp-algo2/k:20-vdb-diag-M6,1.28±0.12\%,87.779656,87.180413
6,mlp-algo2/k:5-vdb-diag-M1,1.41±0.09\%,91.709762,90.060410
7,mlp-algo2/k:5-vdb-diag-M12,1.41±0.20\%,92.987132,92.820567
8,mlp-algo2/k:5-vdb-diag-M6,1.27±0.06\%,85.284991,84.700431
9,mlp-oneshot-vdb-diag-M1,1.46±0.11\%,100.000000,100.000000


In [132]:
table_beta(
    "../artifacts-server/mpg-server/experiment-mnist-256d-test-robustness-beta05/*/*",
    "../artifacts-server/mpg-server/experiment-mnist-256d-test-robustness-vib-beta05/*/*"
)

beta 1e-05
\begin{tabular}{lllrr}
\toprule
{} &                          key &     mean_std &    mCe &  rel_mCe \\
\midrule
0  &   mlp-algo2/k:10-vdb-diag-M1 &  1.32±0.11\% &  84.64 &    85.98 \\
1  &  mlp-algo2/k:10-vdb-diag-M12 &  1.17±0.13\% &  85.44 &    91.73 \\
2  &   mlp-algo2/k:10-vdb-diag-M6 &  1.30±0.10\% &  84.85 &    88.28 \\
3  &   mlp-algo2/k:20-vdb-diag-M1 &  1.37±0.05\% &  88.41 &    89.59 \\
4  &  mlp-algo2/k:20-vdb-diag-M12 &  1.22±0.09\% &  85.60 &    90.95 \\
5  &   mlp-algo2/k:20-vdb-diag-M6 &  1.27±0.04\% &  83.23 &    86.66 \\
6  &    mlp-algo2/k:5-vdb-diag-M1 &  1.43±0.15\% &  89.97 &    92.41 \\
7  &   mlp-algo2/k:5-vdb-diag-M12 &  1.28±0.03\% &  85.62 &    90.00 \\
8  &    mlp-algo2/k:5-vdb-diag-M6 &  1.25±0.08\% &  82.89 &    86.89 \\
9  &      mlp-oneshot-vdb-diag-M1 &  1.63±0.44\% & 100.00 &   100.00 \\
10 &     mlp-oneshot-vdb-diag-M12 &  1.27±0.08\% &  88.64 &    93.77 \\
11 &      mlp-oneshot-vdb-diag-M6 &  1.29±0.02\% &  84.51 &    87.44 \\
12 &     mlp

,key,mean_std,mCe,rel_mCe
0,mlp-algo2/k:10-vdb-diag-M1,1.32±0.11\%,84.642158,85.982204
1,mlp-algo2/k:10-vdb-diag-M12,1.17±0.13\%,85.441649,91.733623
2,mlp-algo2/k:10-vdb-diag-M6,1.30±0.10\%,84.851911,88.275762
3,mlp-algo2/k:20-vdb-diag-M1,1.37±0.05\%,88.407035,89.589473
4,mlp-algo2/k:20-vdb-diag-M12,1.22±0.09\%,85.601826,90.950989
5,mlp-algo2/k:20-vdb-diag-M6,1.27±0.04\%,83.231448,86.663063
6,mlp-algo2/k:5-vdb-diag-M1,1.43±0.15\%,89.973683,92.408634
7,mlp-algo2/k:5-vdb-diag-M12,1.28±0.03\%,85.616232,89.998040
8,mlp-algo2/k:5-vdb-diag-M6,1.25±0.08\%,82.886183,86.892524
9,mlp-oneshot-vdb-diag-M1,1.63±0.44\%,100.000000,100.000000
